In [43]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from math import pi

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import Aer


q = QuantumRegister(3, 'q')
c = ClassicalRegister(3, 'c')
qc = QuantumCircuit(q, c)

qc.rx(pi/2, q[0])
qc.rx(pi/2, q[0]).inverse()
qc.h(q[0])
qc_inv = qc.inverse()
qc_inv.draw('text')


┌───┐┌─────────┐┌──────────┐
q_0: ┤ H ├┤ Rx(π/2) ├┤ Rx(-π/2) ├
     └───┘└─────────┘└──────────┘
q_1: ────────────────────────────
                                 
q_2: ────────────────────────────
                                 
c: 3/════════════════════════════

the resulting quantum circuit essentially consists only of the H gate, since the rotations cancel each other out. There are no other operations applied to the other qbits. Worth noting that the original circuit was inversed, but the H gate stayed the same, since it is its own inverse.

In [71]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

def interpretCounts(counts):
  outputKey = list(counts.keys())[0]
  outputBinary = ("".join([outputKey[7], outputKey[4], outputKey[1],  outputKey[0]]))[::-1]
  return int(outputBinary, 2)


def buildCircuit(inputA, inputB):
  indices = [1, 2, 4, 5, 7, 8]

  q = QuantumRegister(10, 'q')
  c = ClassicalRegister(10, 'c')
  qc = QuantumCircuit(q, c)

  def toBinary(num):
    binary_str = bin(num)[2:]
    binary_str = binary_str.zfill(3)
    return [int(bit) for bit in binary_str]

  def combineSummands(numA, numB):
    acc = []
    for (index, _) in enumerate(numA):
      acc.append(numA[-1 * index -1])
      acc.append(numB[-1 * index -1])

    return acc

  def applyCarry(s_index):
    # CARRY
    qc.ccx(q[s_index + 1], q[s_index + 2], q[s_index + 3])
    qc.cx(q[s_index + 1], q[s_index + 2])
    qc.ccx(q[s_index], q[s_index + 2], q[s_index + 3])

  def applySum(s_index):
    # SUM
    qc.cx(q[s_index+ 1], q[s_index + 2])
    qc.cx(q[s_index], q[s_index + 2])


  #------------------------
  input = combineSummands(toBinary(inputA), toBinary(inputB))
  #------------------------


  qc.reset(q[0])
  qc.reset(q[3])
  qc.reset(q[6])
  qc.reset(q[9])

  for (index, value) in enumerate(input):
    actual_index = indices[index]
    if value == 1:
      qc.reset(q[actual_index])
      qc.x(q[actual_index])
    else:
      qc.reset(q[actual_index])

  applyCarry(0)
  applyCarry(3)
  applyCarry(6)

  qc.cx(q[7], q[8])

  applySum(6)
  applyCarry(3)
  applySum(3)
  applyCarry(0)
  applySum(0)

  qc.measure(q, c)
  # print(qc)
  return qc

In [77]:

from qiskit_aer import Aer

for i in range(0, 8):
  for j in range(0, 8):
    backend = Aer.get_backend('qasm_simulator', shots=1000)

    qc = buildCircuit(i, j)
    job = backend.run(qc)
    counts = job.result().get_counts(qc)

    print(f"{i} + {j} = {interpretCounts(counts)} correct = {i + j == interpretCounts(counts)}")


0 + 0 = 0 correct = True
0 + 1 = 1 correct = True
0 + 2 = 2 correct = True
0 + 3 = 3 correct = True
0 + 4 = 4 correct = True
0 + 5 = 5 correct = True
0 + 6 = 6 correct = True
0 + 7 = 7 correct = True
1 + 0 = 1 correct = True
1 + 1 = 2 correct = True
1 + 2 = 3 correct = True
1 + 3 = 4 correct = True
1 + 4 = 5 correct = True
1 + 5 = 6 correct = True
1 + 6 = 7 correct = True
1 + 7 = 8 correct = True
2 + 0 = 2 correct = True
2 + 1 = 3 correct = True
2 + 2 = 4 correct = True
2 + 3 = 5 correct = True
2 + 4 = 6 correct = True
2 + 5 = 7 correct = True
2 + 6 = 8 correct = True
2 + 7 = 9 correct = True
3 + 0 = 3 correct = True
3 + 1 = 4 correct = True
3 + 2 = 5 correct = True
3 + 3 = 6 correct = True
3 + 4 = 7 correct = True
3 + 5 = 8 correct = True
3 + 6 = 9 correct = True
3 + 7 = 10 correct = True
4 + 0 = 4 correct = True
4 + 1 = 5 correct = True
4 + 2 = 6 correct = True
4 + 3 = 7 correct = True
4 + 4 = 8 correct = True
4 + 5 = 9 correct = True
4 + 6 = 10 correct = True
4 + 7 = 11 correct = Tr

In [75]:


print(buildCircuit(0, 0))



                                                                            »
 q_0: ─|0>─────────────■────────────────────────────────────────────────────»
                       │                                                    »
 q_1: ─|0>───■────■────┼────────────────────────────────────────────────────»
             │  ┌─┴─┐  │                                                    »
 q_2: ─|0>───■──┤ X ├──■────────────────────────────────────────────────────»
           ┌─┴─┐└───┘┌─┴─┐                                                  »
 q_3: ─|0>─┤ X ├─────┤ X ├──■──────────────────────────────────■─────────■──»
           └───┘     └───┘  │                                  │         │  »
 q_4: ─|0>───■────■─────────┼────────────────────────■────■────┼────■────┼──»
             │  ┌─┴─┐       │                        │  ┌─┴─┐  │  ┌─┴─┐┌─┴─┐»
 q_5: ─|0>───■──┤ X ├───────■────────────────────────■──┤ X ├──■──┤ X ├┤ X ├»
           ┌─┴─┐└───┘     ┌─┴─┐                    ┌─┴─┐└───┘┌─┴